# Cats vs Dogs

In [1]:
import sys
sys.path.insert(0, './../..')
from importlib import reload
import utils; reload(utils)
from utils import *

K.set_image_dim_ordering('th')
batch_size=64
output_size = 2
#base_dir = '../
base_dir = '../sample/'
dropout = 0.5

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [ ]:
gen = image.ImageDataGenerator()
gen_with_aug = image.ImageDataGenerator(rotation_range=10, 
                                        width_shift_range=0.05, 
                                        zoom_range=0.05, 
                                        channel_shift_range=10, 
                                        height_shift_range=0.05, 
                                        shear_range=0.05, 
                                        horizontal_flip=True)

batches = gen.flow_from_directory(base_dir + 'train/', 
                                  target_size=(224,224),
                                  class_mode='categorical', 
                                  shuffle=False, 
                                  batch_size=batch_size)
batches_aug = gen_with_aug.flow_from_directory(base_dir + 'train/', 
                                                   target_size=(224,224),
                                                   class_mode='categorical', 
                                                   shuffle=True, 
                                                   batch_size=batch_size)
val_batches = gen.flow_from_directory(base_dir + 'valid/',
                                     target_size=(224,224),
                                     class_mode='categorical',
                                     shuffle=False,
                                     batch_size=batch_size)
test_batches = gen.flow_from_directory(base_dir + 'test/',
                                     target_size=(224,224),
                                     class_mode='categorical',
                                     shuffle=False,
                                     batch_size=batch_size*2)

trn_labels = to_categorical(batches.classes)
val_labels = to_categorical(val_batches.classes)

## Finetuning

In [ ]:
def finetune_last_layer(model, index):
    log_action("Training last layer")
    dm = vgg_fc_model(model, output_size, dropout=0.5)
    for i in range(len(dm.layers)): dm.layers[i].trainable = i >= len(dm.layers) - 3
    
    fit_with_features(dm, RMSprop(1e-5), 12, trn_features, trn_labels, val_features, val_labels, batch_size=batch_size)
    fit_with_features(dm, RMSprop(1e-7), 6, trn_features, trn_labels, val_features, val_labels, batch_size=batch_size)
    
    for l1,l2 in zip(model.layers[last_conv_idx(model) + 1:], dm.layers): l1.set_weights(l2.get_weights())
    model.save_weights(base_dir + 'models/last' + str(index) + '.h5')
    
def finetune_dense_layers(model, index):
    log_action("Training dense layers")
    dm = vgg_fc_model(model, output_size, dropout=0.5)
    for l in dm.layers: l.trainable = True
    
    fit_with_features(dm, RMSprop(1e-5), 10, trn_features, trn_labels, val_features, val_labels, batch_size=batch_size)
    fit_with_features(dm, RMSprop(1e-6), 8, trn_features, trn_labels, val_features, val_labels, batch_size=batch_size)
    fit_with_features(dm, RMSprop(1e-7), 10, trn_features, trn_labels, val_features, val_labels, batch_size=batch_size)
    
    for l1,l2 in zip(model.layers[last_conv_idx(model) + 1:], dm.layers): l1.set_weights(l2.get_weights())
    model.save_weights(base_dir + 'models/dense' + str(index) + '.h5')
    
def finetune_dense_layers_with_aug(model, index):
    log_action("Training dense layers with augmentations")
    for i in range(len(model.layers)): model.layers[i].trainable = i >= 16
    fit_dense_layers_with_aug(model, index, RMSprop(1e-5), 8)
    fit_dense_layers_with_aug(model, index, RMSprop(1e-7), 10)
    
def fit_dense_layers_with_aug(model, index, lr, epochs):
    fit_with_batches(model, lr, epochs, batches_aug, val_batches, batch_size=batch_size)
    model.save_weights(base_dir + 'models/dense_aug' + str(index) + '.h5')

## Cache convolutional output

In [ ]:
conv_model = conv_model(vgg(output_size, dropout=dropout))

batches.reset()
trn_features = conv_model.predict_generator(batches, steps(batches, batch_size))
save_array(base_dir + 'models/train_convlayer_features.bc', trn_features)

val_batches.reset()
val_features = conv_model.predict_generator(val_batches, steps(batches, batch_size))
save_array(base_dir + 'models/valid_convlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(base_dir + 'models/train_convlayer_features.bc')
val_features = load_array(base_dir + 'models/valid_convlayer_features.bc')

## Train models

In [ ]:
clear_logs()
for i in range(5):
    model = vgg(output_size, dropout=dropout)
    finetune_last_layer(model, i)
    finetune_dense_layers(model, i)
    finetune_dense_layers_with_aug(model, i)
log_action("finished training")

## Ensembling

In [ ]:
ens_preds = ensemble_predict(vgg(output_size, dropout=dropout), 'models/dense_aug', val_batches, size=5)
print(categorical_accuracy(val_labels, ens_preds).eval())

## Submit to Kaggle

In [ ]:
# Compute predictions
preds = ensemble_predict(vgg(output_size, dropout=dropout), 'models/dense_aug', test_batches, size=5)

# Clip predictions
isdog = preds[:,1].clip(min=0.020, max=0.980) 

# Extract image ids
filenames = batches.filenames
ids = np.array([int(f[6:f.find('.')]) for f in filenames])

# Write to CSV file
subm = np.stack([ids,isdog], axis=1)
np.savetxt('../sample_submission.csv', subm, fmt='%d,%.5f', header='id,label', comments='')

# Get link
from IPython.display import FileLink
FileLink('../sample_submission.csv')